# Stability calculations

## -- LUVOIR --

In notebook 10, we calculated the PASTIS modes for all three currently designed apodizers for the LUVOIR A pupil. In this notebook, we will load the modes and calculate the maximum mode contributions $\sigma$ and strability requirements $\Delta \sigma$.

After that I also calculate the maximum segment contributions $\mu$.

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy as hc

os.chdir('../../pastis/')
import util_pastis as util
from e2e_simulators.luvoir_imaging_new import LuvoirAPLC

In [ ]:
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Computer Modern Sans"]
plt.rcParams["text.usetex"] = True
plt.rcParams["text.latex.preamble"] = r"\usepackage{cmbright}"
plt.rcParams['font.size'] = 16
plt.rcParams['axes.labelsize'] = 22
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['figure.titlesize'] = 16

## Instantiate a simple SM for mode plotting

I need to simplify this... I can make a child class like I did for the full LUVOIR simulator.

In [ ]:
# Load aperture files needed for SM
nseg = 120
wvln = 638e-9

datadir = '/Users/pueyo/PythonPackages/PASTIS/LUVOIR_delivery_May2019/'
aper_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000.fits'
aper_ind_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000_indexed.fits'
aper_read = hc.read_fits(os.path.join(datadir, aper_path))
aper_ind_read = hc.read_fits(os.path.join(datadir, aper_ind_path))

pupil_grid = hc.make_pupil_grid(dims=aper_ind_read.shape[0], diameter=15)
aper = hc.Field(aper_read.ravel(), pupil_grid)
aper_ind = hc.Field(aper_ind_read.ravel(), pupil_grid)

wf_aper = hc.Wavefront(aper, wvln)

# Load segment positions from fits header
hdr = fits.getheader(os.path.join(datadir, aper_ind_path))

poslist = []
for i in range(nseg):
    segname = 'SEG' + str(i+1)
    xin = hdr[segname + '_X']
    yin = hdr[segname + '_Y']
    poslist.append((xin, yin))
    
poslist = np.transpose(np.array(poslist))
seg_pos = hc.CartesianGrid(hc.UnstructuredCoords(poslist))

## Mode and eigenvalue units

Since the matrices got generated normalized to an aberration of 1 nm, everything stemming from the matrix will be in units of **nanometers**. --> This is the desired behaviour of course and is yet to be confirmed, which is one of the goals of this notebook.

In [ ]:
eunit = 1e-9

## Also instantiate a LUVOR telescope

For cumulative contrast calculation.

> **ATTENTION:** the variable `apodizer_design` has to be correct for the data we are reading with the variable `savedpath` further below!!!

In [ ]:
savedpath = '/Users/pueyo/Documents/data_from_repos/pastis_data/2020-06-22T13-38-29_luvoir-medium'   # large apodizer LUVOIR
matrix = fits.getdata(os.path.join(savedpath, 'matrix_numerical', 'PASTISmatrix_num_Multiple_Modes_MaxZer_6.fits'))
plt.figure(figsize=(10, 10))
plt.imshow(matrix)
plt.colorbar()

In [ ]:
# Instantiate LUVOIR
sampling = 4
apodizer_design = 'medium'
# This path is specific to the paths used in the LuvoirAPLC class
optics_input = '/Users/pueyo/PythonPackages/PASTIS/LUVOIR_delivery_May2019/'
luvoir = LuvoirAPLC(optics_input, apodizer_design, sampling)
luvoir.make_segment_zernike_primary(6)

In [ ]:
zernike_coeffs = np.zeros([6*120])
zernike_coeffs[4] = eunit
luvoir.sm.actuators = zernike_coeffs
wf_sm = luvoir.sm(luvoir.wf_aper)
plt.figure(figsize=(10, 10))
hc.imshow_field(wf_sm.phase)

In [ ]:
zernike_coeffs = np.zeros([6*120])
zernike_coeffs[4] = 0
luvoir.sm.actuators = zernike_coeffs
wf_sm = luvoir.sm(luvoir.wf_aper)
psf_unaber, ref = luvoir.calc_psf(ref=True)
norm = ref.max()

In [ ]:
psf_unaber, ref = luvoir.calc_psf(ref=True)
norm = ref.max()
plt.figure(figsize=(10, 10))
hc.imshow_field(psf_unaber, norm=LogNorm())

In [ ]:
# Make dark hole
dh_outer = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['owa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_inner = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['iwa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_mask = (dh_outer - dh_inner).astype('bool')

In [ ]:
dh_intensity = psf_unaber/norm * dh_mask
baseline_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])
print('contrast:', baseline_contrast)

## Read eigenmodes and eigenvalues

In [ ]:
evals, evecs = np.linalg.eig(matrix)
sorted_evals = np.sort(evals)
sorted_indices = np.argsort(evals)
sorted_evecs = evecs[:, sorted_indices]
plt.figure(figsize=(14, 8))
#plt.plot(evals, label='Unsorted from eigendecomposition')
plt.plot(sorted_evals, label='Sorted lowest to highest evals')
plt.semilogy()
plt.xlabel('Eigenmodes')
plt.ylabel('Log Eigenvalues')
plt.legend()

In [ ]:
# Lets compute all modes now for LUVOIR with an HCIPy SM
emodes = []

for mode in range(len(evals)):
    print('Working on mode {}/{}.'.format(mode+1, len(evals)))
    
    zernike_coeffs = eunit*sorted_evecs[:, mode]/2
    luvoir.sm.actuators = zernike_coeffs
    wf_sm = luvoir.sm(luvoir.wf_aper)
    emodes.append(wf_sm.phase)

## Stability requirements

### Static contrast and static contribution

According to Lucie's paper, we can get the maximum aberration $\sigma_p$ we can allow per mode $p$ if we want to obtain a contrast $C$ directly from the according eigenvalue $\lambda_p$.

$$\sigma_p = \sqrt{\frac{C_p}{\lambda_p}}$$

Where $C_p$ is the contrast contribution from mode $p$ only. For simplicity, we assume that all modes have the same contribution to the total contrast $C$, meaning

$$C = C_1 + C_2 + ... + C_N$$

with $N$ being the total number of modes. This also means

$$C_p = \frac{C}{N}$$

Since we have one mode that has a really low eigenvalue and essentially on contribution, we can take that mode out and assume that the other $N-1$ modes bear all the contrast contributions, changing it to

$$C_p = C_1 + C_2 + ... + C_{N-1}$$

and

$$C_p = \frac{C}{N-1}$$

So we can calculate the maximum ocntribution of a mode $\sigma_p$ with

$$\sigma_p = \sqrt{\frac{C}{(N-1)\lambda_p}}$$

We have 120 segments, so $N=120$ and we will aim to achieve a static contrast of $C = 10^{-10}$.

Note how we start numbering at 1 here, to stay consistent with our segment numbering, but when doing these things in Python we need to start numbering at 0.

Skype with Lucie, have to include baseline contrast $C_0$:

$$\sigma_p = \sqrt{\frac{C-C_0}{(N-1)\lambda_p}}$$

In [ ]:
number_of_modes = luvoir.sm.num_actuators

In [ ]:
c_stat = 1e-11
print('Static contrast: {}'.format(c_stat))
print('N = {}'.format(number_of_modes))

In [ ]:
# Calculate single sigma - remember that we start numbering at 0 because of python
def get_sigma(cstat, nseg, eigenval, c_zero):
#     sigma = np.sqrt((cstat - c_zero) / ((nseg-1)*eigenval))
    sigma = np.sqrt((cstat) / ((nseg-1)*eigenval))

    return sigma

In [ ]:
p = 1
sigma_p = get_sigma(c_stat, number_of_modes, sorted_evals[p], baseline_contrast)
print(sigma_p)

In [ ]:
# Do them all at once
sigmas = get_sigma(c_stat, number_of_modes, sorted_evals, baseline_contrast)
#print(sigmas)

# Save them
#np.savetxt(os.path.join(savedpath, 'results', 'sigmas.txt'), sigmas)

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(sigmas[1:])
plt.semilogy()
plt.title('Constraints per mode', size=15)
plt.xlabel('Mode', size=15)
plt.ylabel('Max mode contribution $\sigma_p$ (nm)', size=15)

### Dynamic contast and dynamic contribution

We want $\Delta C = 10^{-11}$. This "dynamic contast" is the error on the contrast $C$ which we want to limit, so we write it as

$$\Delta C = \sqrt{\Delta C_1^2 + \Delta C_2^2 + ... \Delta C_N^2} = \sqrt{N \Delta C_p^2} = \sqrt{N} \Delta C_p$$

which also means

$$\Delta C_p = \frac{C}{\sqrt{N}}$$

and then again because we discard that non-contributing mode, we actually have

$$\Delta C_p = \frac{C}{\sqrt{N-1}}$$

For the $\Delta \sigma_p$ we can use the same equation like for the $\sigma_p$, but we plug in $\Delta C$ instead of $C$.

$$\Delta \sigma_p = \sqrt{\frac{\Delta C}{\lambda_p}} = \sqrt{\frac{\Delta C}{\sqrt{(N-1)}\ \lambda_p}}$$

In [ ]:
c_dyn = 1e-11

# Calculate the Delta Cs
def get_delta_sigma(cdyn, nseg, eigenval):
    del_sigma = np.sqrt(cdyn / (np.sqrt(nseg-1)*eigenval))
    return del_sigma

In [ ]:
del_sigmas = get_delta_sigma(c_dyn, number_of_modes, sorted_evals)
#print(del_sigmas)

# Save them
#np.savetxt(os.path.join(savedpath, 'results', 'delta_sigmas.txt'), del_sigmas)

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(del_sigmas[1:])
plt.semilogy()
plt.title('Stability per mode', size=15)
plt.xlabel('Mode', size=15)
plt.ylabel('Max mode contribution $\Delta \sigma_p$ (nm)', size=15)

These are only slightly larger than the $\sigma_p$.

### Cumulative contrast plot

I guess the only way I can think of right now to verify at least the $\sigma_p$ is to make the same cumulative contrast like Fig. 11b in Lucie's paper. Let's do that.

Since I never get rid of the first mode, global piston with a ridiculously low eigenvalue, I will have a `Nan` in the sigma array, so when I sum up the OPD, I have to use `numpy.nansum()` instead of `numpy.nan()`.

#### Cumulative contrast with E2E simulator

In [ ]:
eunit = 1e-9

In [ ]:
cont_cum_e2e = np.zeros([len(evals)])
cont_stick_e2e = np.zeros([len(evals)])
tmp_cont = 0 
for maxmode in range(len(evals)):
#for maxmode in range(40):

    print(maxmode)
    zernike_coeffs = sigmas[maxmode]*sorted_evecs[:,maxmode]
    luvoir.sm.actuators = eunit*zernike_coeffs/2
    luvoir.sm(luvoir.wf_aper)

    # Get PSF from putting this OPD on the SM
    psf = luvoir.calc_psf()

    # Calculate the contrast from that PSF
    dh_intensity = psf/norm * dh_mask
    contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)]) - baseline_contrast
    cont_cum_e2e[maxmode] = contrast
    tmp_cont = tmp_cont+contrast
    cont_stick_e2e[maxmode] = tmp_cont

In [ ]:
# Save cumulative contrast plot from E2E simulator
#np.savetxt(os.path.join(savedpath, 'results', 'cumulative_contrast_e2e.txt'), cont_cum_e2e)

In [ ]:
# Plot the cumulative contrast from E2E simulator
plt.figure(figsize=(16, 10))
plt.plot(cont_stick_e2e)
plt.title('E2E cumulative contrast for target $C$ = ' + str(c_stat), size=15)
plt.xlabel('Mode number', size=15)
plt.ylabel('Constrast', size=15)

#plt.savefig(os.path.join(savedpath, 'results', 'cumulative_contrast_e2e.pdf'))

#### Cumulative cotrast with numerical PASTIS matrix

In [ ]:
# Read matrix
#matrix = fits.getdata(os.path.join(savedpath, 'matrix_numerical/PASTISmatrix_num_piston_Noll1.fits'))

# Calculate cumulative contrast
cont_cum_pastis = np.zeros([len(evals)])
cont_stick_pastis = np.zeros([len(evals)])
tmp_cont = 0 
for maxmode in range(len(evals)):
    aber = sorted_evecs[:,maxmode]*sigmas[maxmode]
    aber *= u.nm
    contrast_matrix = util.pastis_contrast(aber, matrix)
    cont_cum_pastis[maxmode] = (contrast_matrix)
    tmp_cont = tmp_cont+contrast_matrix
    cont_stick_pastis[maxmode] = tmp_cont

In [ ]:
# Save cumulative contrast plot from numerical PASTIS matrix
#np.savetxt(os.path.join(savedpath, 'results', 'cumulative_contrast_pastis.txt'), cont_cum_pastis)

In [ ]:
plt.figure(figsize=(16, 10))
plt.plot(cont_cum_e2e, label='E2E')
plt.plot(cont_cum_pastis, label='PASTIS')
plt.title('Cumulative contrast for target $C$ = ' + str(c_stat), size=15)
plt.xlabel('Mode number', size=15)
plt.ylabel('Constrast', size=15)
plt.legend()

In [ ]:
plt.figure(figsize=(16, 10))
plt.plot(cont_stick_e2e, label='E2E')
plt.plot(cont_stick_pastis, label='PASTIS')
plt.title('Cumulative contrast for target $C$ = ' + str(c_stat), size=15)
plt.xlabel('Mode number', size=15)
plt.ylabel('Constrast', size=15)
plt.legend()

## Checking normalisation

$M$ is a mode and $p,q$ go from 1 to 120. The modes are normalized when their dot product is 1 for a mode with itself and 0 in all other cases:

$$<M_p, M_q>_{pup} = \begin{cases} \mbox{0} & p \neq q \\ \mbox{1} & p = q \end{cases}$$

The image plane modes $\mathscr{C}(M_p)$ (= a pupil plane mode $M$ propagated through the coronagraph) should be orthogonal and the diagonal of this matrix filled with the eigenvaues $\Lambda_p$:

$$<\mathscr{C}(M_p), \mathscr{C}(M_q)>_{DH} = \begin{cases} \mbox{0} & p \neq q \\ \Lambda_p & p = q \end{cases}$$

### Check orthonormality for pupil modes $M_p$

In [ ]:
# Try this for a random pair of modes
modedot1 = 4
modedot2 = 100
res = np.dot(emodes[modedot1], emodes[modedot2])
print(res)

In [ ]:
# Do this for all modes
pupildot = np.zeros((6*120, 6*120))

for dotrun in range(len(evals)):
    print(dotrun)
    for dotforrest in range(len(evals)):
        pupildot[dotrun, dotforrest] = np.dot(emodes[dotrun], emodes[dotforrest])

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(pupildot)

In [ ]:
plt.plot(pupildot[100,:])

In [ ]:
pupildot2 = np.zeros((6*120, 6*120))

for dotrun in range(len(evals)):
    print(dotrun)
    for dotforrest in range(len(evals)):
        pupildot2[dotrun, dotforrest] = np.dot(sorted_evecs[:,dotrun], sorted_evecs[:,dotforrest])

In [ ]:
plt.figure(figsize=(10, 10))
plt.imshow(pupildot2)

In [ ]:
plt.plot(pupildot2[1,:])

### Create or read image modes $\mathscr{C}(M_p)$

In [ ]:
# Create them
all_psfs = []
all_efields = []

for maxmode in range(len(evals)):
#for maxmode in range(40):

    print(maxmode)
    zernike_coeffs = sorted_evecs[:,maxmode]
    luvoir.sm.actuators = eunit*zernike_coeffs/2
    luvoir.sm(luvoir.wf_aper)
    
    psf, ref, inter = luvoir.calc_psf(ref=True, return_intermediate='efield')
    # Everything gets returns as an E-field here, the coro PSF, the no-FPM reference and the intermediate planes.
    # Except for the intensity image right after the FPM.
    
    # Save as fits
    #hc.write_fits(psf, os.path.join(savedpath, 'results', 'modes', 'image_modes', 'mode'+str(gump+1)+'.fits'))
    
    all_psfs.append(psf)
    all_efields.append(inter)

In [ ]:
# Read them
# ...

In [ ]:
# Check the intermediates
print(type(all_efields))
print(type(all_efields[0]))
print(all_efields[0].keys())
print(type(all_efields[0]['after_lyot']))

In [ ]:
# Check one of the PSFs
print(len(all_psfs))
plt.figure(figsize=(10, 10))
hc.imshow_field(all_psfs[3].intensity/norm, norm=LogNorm())
plt.colorbar()

### Checking the orthogonality for image modes $\mathscr{C}(M_p)$

In [ ]:
zernike_coeffs = np.zeros([6*120])
luvoir.sm.actuators = zernike_coeffs
wf_sm = luvoir.sm(luvoir.wf_aper)
psf_unaber, ref, inter_unaber = luvoir.calc_psf(ref=True, return_intermediate='efield')

In [ ]:
# Try this for a random pair of modes
imdot1 = 479
imdot2 = 100

# test1_field = all_psfs[imdot1].electric_field - psf_unaber.electric_field
# test2_field = all_psfs[imdot2].electric_field  - psf_unaber.electric_field 
test1_field = all_psfs[imdot1].electric_field - psf_unaber.electric_field
test2_field = all_psfs[imdot2].electric_field - psf_unaber.electric_field 
res = np.vdot((test1_field * dh_mask ), (test2_field * dh_mask))/norm
print(res)

In [ ]:
dhdh = np.zeros((6*120, 6*120))
mean_int = []
for dotrun in range(len(evals)):
    print(dotrun)
    inten = np.abs(all_psfs[dotrun].electric_field - psf_unaber.electric_field)**2/norm * dh_mask
    mean_int.append(np.mean(inten[np.where(inten != 0)]))
    
    for dotforrest in range(len(evals)):
        test1_field = all_psfs[dotrun].electric_field - psf_unaber.electric_field
        test2_field = all_psfs[dotforrest].electric_field - psf_unaber.electric_field 
        res = np.vdot((test1_field * dh_mask ), (test2_field * dh_mask))/norm
        dhdh[dotrun, dotforrest] = res

In [ ]:
# Plot the mean contrast in the DH as function of mode
plt.plot(mean_int)
plt.plot(sorted_evals)
plt.semilogy()
plt.ylabel('Mean contrast in DH')
plt.xlabel('Mode number')

In [ ]:
print(dhdh.shape)
plt.figure(figsize=(10, 10))
plt.imshow(dhdh)
plt.title('Image plane orthogonality')
plt.colorbar()

So this part dos not work proprly yet. We need to check is a little more. I believe the problem is the raw contrast not being subtractd at the right place

## Segment based constraints

In [ ]:
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams["font.sans-serif"] = ["Computer Modern Sans"]
plt.rcParams["text.usetex"] = True
plt.rcParams["text.latex.preamble"] = r"\usepackage{cmbright}"
plt.rcParams['font.size'] = 16
plt.rcParams['axes.labelsize'] = 22
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['figure.titlesize'] = 16

In [ ]:
c_target = 1e-10

In [ ]:
number_of_modes = luvoir.sm.num_actuators
# Calculate the inverse of the pastis MODE matrix
modestosegs = np.linalg.pinv(sorted_evecs)

# # Calculate all mean contrasts of the pastis modes directly (as-is, with natural normalization)
# c_avg = []
# for i in range(number_of_modes):
#     c_avg.append(util.pastis_contrast(sorted_evecs[:, i] * u.nm, matrix) + baseline_contrast)

# # Calculate segment requirements
# mu_map = np.sqrt(
#     ((c_target - baseline_contrast) / number_of_modes) / (np.dot(c_avg - baseline_contrast, np.square(modestosegs))))

# Calculate all mean contrasts of the pastis modes directly (as-is, with natural normalization)
c_avg = []
for i in range(number_of_modes):
    c_avg.append(util.pastis_contrast(sorted_evecs[:, i] * u.nm, matrix) )

# Calculate segment requirements
mu_map = np.sqrt(
    ((c_target ) / number_of_modes) / (np.dot(c_avg, np.square(modestosegs))))

In [ ]:
zernike_coeffs = mu_map*eunit/2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref, inter = luvoir.calc_psf(ref=True, return_intermediate='efield')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf.intensity/norm, norm=LogNorm())
plt.colorbar()
dh_intensity = np.abs(psf.electric_field - psf_unaber.electric_field)**2/norm 
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])
print('contrast:', test_contrast)

In [ ]:
N_zernike = 6
zernike_coeffs_numaps = np.zeros([N_zernike,number_of_modes])
for qq in range(N_zernike):
    zernike_coeffs_tmp = np.zeros([number_of_modes])
    for pp in range(120):
        zernike_coeffs_tmp[qq+(pp)*N_zernike] = mu_map[qq+(pp)*N_zernike]
    zernike_coeffs_numaps[qq] = zernike_coeffs_tmp

In [ ]:
nu_maps = []
for qq in range(N_zernike):
    zernike_coeffs = eunit*zernike_coeffs_numaps[qq]/2
    luvoir.sm.actuators = zernike_coeffs
    nu_maps.append(luvoir.sm(luvoir.wf_aper).phase/luvoir.wf_aper.wavenumber)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps[0]/eunit*1000, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level Piston ')

In [ ]:
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps[0]/eunit*1000, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level Piston ')
zernike_coeffs = zernike_coeffs_numaps[0]*eunit/2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref = luvoir.calc_psf(ref=True, return_intermediate='False')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf/norm, norm=LogNorm())
plt.colorbar()
dh_intensity = psf/norm * dh_mask
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])- baseline_contrast
print('contrast:', test_contrast)

In [ ]:
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps[1]/eunit*1000, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level Tip ')
zernike_coeffs = zernike_coeffs_numaps[1]*eunit/2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref = luvoir.calc_psf(ref=True, return_intermediate='False')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf/norm, norm=LogNorm())
plt.colorbar()
dh_intensity = psf/norm * dh_mask
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)]) - baseline_contrast
print('contrast:', test_contrast)

In [ ]:
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps[2]/eunit*1000, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level tilt ')
zernike_coeffs = zernike_coeffs_numaps[2]*eunit / 2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref = luvoir.calc_psf(ref=True, return_intermediate='False')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf/norm, norm=LogNorm())
plt.colorbar()
dh_intensity = psf/norm * dh_mask
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)]) - baseline_contrast
print('contrast:', test_contrast)

In [ ]:
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps[3]/eunit*1000, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segmement Level focus ')
zernike_coeffs = zernike_coeffs_numaps[3]*eunit/2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref = luvoir.calc_psf(ref=True, return_intermediate='False')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf/norm, norm=LogNorm())
plt.colorbar()
dh_intensity = psf/norm * dh_mask
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)]) - baseline_contrast
print('contrast:', test_contrast)

In [ ]:
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps[4]/eunit*1000, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segmement Level twisted focus 1 ')
zernike_coeffs = zernike_coeffs_numaps[3]*eunit/2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref = luvoir.calc_psf(ref=True, return_intermediate='False')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf/norm, norm=LogNorm())
plt.colorbar()
dh_intensity = psf/norm * dh_mask
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)]) - baseline_contrast
print('contrast:', test_contrast)

In [ ]:
fig, ax = plt.subplots(ncols=1)
img1 = hc.imshow_field(nu_maps[5]/eunit*1000, cmap='RdBu')#, vmin=pmin, vmax=pmax)
clb = fig.colorbar(img1)
clb.set_label('pm', rotation=90)
plt.tight_layout(h_pad=1)
plt.title('Segment Level twisted focus 2 ')
zernike_coeffs = zernike_coeffs_numaps[3]*eunit/2
luvoir.sm.actuators = zernike_coeffs
luvoir.sm(luvoir.wf_aper)
psf, ref = luvoir.calc_psf(ref=True, return_intermediate='False')
plt.figure(figsize=(10, 10))
hc.imshow_field(psf/norm, norm=LogNorm())
plt.colorbar(aspect = 4)
dh_intensity = psf/norm * dh_mask
test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)]) - baseline_contrast
print('contrast:', test_contrast)

In [ ]:
n_repeat = 3000
all_contr_rand_seg = []
all_random_maps = []
for rep in range(n_repeat):
    print('Segment realization {}/{}'.format(rep + 1, n_repeat))
    zernike_coeffs = np.random.uniform(-mu_map*eunit,mu_map*eunit,number_of_modes)
    luvoir.sm.actuators = zernike_coeffs
    luvoir.sm(luvoir.wf_aper)
    psf, ref, inter = luvoir.calc_psf(ref=True, return_intermediate='efield')
#     plt.figure(figsize=(10, 10))
#     hc.imshow_field(psf.intensity/norm, norm=LogNorm())
#     plt.colorbar()
    dh_intensity = np.abs(psf.electric_field - psf_unaber.electric_field)**2/norm 
    test_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])
    all_contr_rand_seg.append(test_contrast) 
    print('contrast:', test_contrast)
    

In [ ]:
plt.figure(figsize=(16, 10))
plt.hist(all_contr_rand_seg,30)
plt.title('E2E raw contrast, {} realizations, target contrast 1e-10'.format(n_repeat), size=20)
plt.xlabel('Mean contrast in DH', size=20)
plt.ylabel('PDF', size=20)
plt.tick_params(axis='both', which='both', length=6, width=2, labelsize=25)